In [1]:

from ytmusicapi import YTMusic
from pprint import pprint
import os
from pytube import YouTube
import urllib.request
import requests
import re
import string
from tkinter import Tk, ttk, filedialog, StringVar
from dotenv import load_dotenv
from spotipy import SpotifyOAuth
from spotipy.oauth2 import SpotifyOauthError
import pandas as pd
import numpy as np
import json

In [42]:
class YTScraper:
  def __init__(
    self,
    token_file_path = 'oauth.json',
  ):
    self.yt_music = YTMusic(token_file_path)
    
  def get_search_results(self,search_str):
    return self.yt_music.search(search_str,filter = 'songs')
  
  def get_lyrics(self, video_id):
    video=self.yt_music.get_watch_playlist(videoId=video_id)
    lyrics_id = video['lyrics']
    lyrics = None
    if lyrics_id:
      lyrics=self.yt_music.get_lyrics(lyrics_id)
    return lyrics
  
  def download_song_by_video_id(
    self,
    video_id,
    output_path=None,
  ):
    if output_path is not None:
      os.makedirs(output_path,exist_ok=True)
    else:
      output_path = ""
    yt = YouTube(f"https://youtube.com/watch?v={video_id}", use_oauth = True, allow_oauth_cache=True)
    temp_file = os.path.join(output_path,f"{video_id}.mp4")
    yt.streams.filter(only_audio=True).first().download(output_path=output_path, filename=f"{video_id}.mp4")
    if os.path.exists(temp_file):
      print(f"Downloaded successfully: {temp_file}")
    return os.path.join(output_path,f"{video_id}.mp4")
  
  def scrap_by_search(self, search_str, download_output_file_path = None):
    search_result = self.get_search_results(search_str=search_str)
    video_id = None
    for item in search_result:
      if item['resultType'] in ['song'] and item['category'] == 'Songs':
        video_id = item['videoId']
        break 
    if (video_id):
      lyrics = self.get_lyrics(video_id=video_id)
      if not lyrics:
        lyrics = {'lyrics':None,'source':None}
      download_path = self.download_song_by_video_id(video_id=video_id, output_path=download_output_file_path)
    else:
      raise Exception("Null Video Id")
    return {
      "lyrics" : lyrics['lyrics'],
      "audio_file_path" : download_path
    }
  

In [17]:
yt_scraper = YTScraper()

In [53]:
yt_scraper.yt_music.search('Washington on your side',filter = 'songs')
print()

NameError: name 'yt_scraper' is not defined

In [19]:
pprint(yt_scraper.scrap_by_search("Washington on your side", download_output_file_path = "songs"))

songs\9014vq1lqXM.mp4
Downloaded successfully: songs\9014vq1lqXM.mp4
{'file_path': 'songs\\9014vq1lqXM.mp4',
 'lyrics': 'It must be nice, it must be nice\r\n'
           'To have Washington on your side\r\n'
           'It must be nice, it must be nice \r\n'
           'To have Washington on your side\r\n'
           '\r\n'
           "Ev'ry action has its equal, opposite reactions\r\n"
           "Thanks to Hamilton, our cab'net's fractured into factions\r\n"
           "Try not to crack under the stress, we're breaking down like "
           'fractions\r\n'
           "We smack each other in the press, and we don't print "
           'retractions\r\n'
           'I get no satisfaction witnessing his fits of passion\r\n'
           'The way he primps and preens and dresses like the pits of '
           'fashion\r\n'
           'Our poorest citizens, our farmers, live ration to ration\r\n'
           "As Wall Street robs 'em blind in search of chips to cash in\r\n"
           "This pri

In [ ]:
r = requests.get(f"https://api-partner.spotify.com/pathfinder/v1/query?operationName=searchDesktop&variables=%7B%22searchTerm%22%3A%22{songName.replace(' ', '+')}%22%2C%22offset%22%3A0%2C%22limit%22%3A10%2C%22numberOfTopResults%22%3A5%2C%22includeAudiobooks%22%3Afalse%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22130115162add6f3499d2f88ead8a37a7cad1d4d2314f3a206377035e7d26b74c%22%7D%7D", headers=headers)

### Loading env variables

### Spotify Scraper Class

In [82]:
class SpotifyScraper:
    def __init__(
        self,
        client_id,
        client_secret,
        redirect_uri,
        scope = "user-library-read playlist-read-private playlist-read-collaborative"
    ):
        self.yt_scraper = YTScraper()
        try:
            self.sp_oauth = SpotifyOAuth(
                client_id=client_id,
                client_secret=client_secret,
                redirect_uri=redirect_uri,
                scope="user-library-read playlist-read-private playlist-read-collaborative",
            )
        except SpotifyOauthError as e:
            print(f"Spotify OAuth setup error: {e}")
            exit(1)
            
        self.access_token = self.get_access_token()
            
    def get_access_token(self):
        # Get access token
        self.token_info = self.sp_oauth.get_cached_token()
        if not self.token_info:
            auth_url = self.sp_oauth.get_authorize_url()
            print("Please go to this URL and authorize the app:", auth_url)
            auth_code = input("Enter the authorization code: ")
            self.token_info = self.sp_oauth.get_access_token(auth_code)
        access_token = self.token_info["access_token"]
        return access_token

    def get_auth_header(self):
        return {"Authorization": "Bearer " + self.access_token}
    # Function to fetch user playlists
    
    def get_user_playlists(self):
        print("Retrieving user playlists...")
        headers = self.get_auth_header()
        response = requests.get("https://api.spotify.com/v1/me/playlists", headers=headers)
        response_json = response.json()
        # for item in response_json["items"]:
        #     playlists[item["name"]] = item["id"]
        print("Playlists retrieved successfully.")
        return response_json
    
    @staticmethod
    def sanitize_filename(filename):
        valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
        return ''.join(c for c in filename if c in valid_chars)

    def get_track_info_by_id(self, track_id):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/tracks/{track_id}", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_playlist_info_by_id(self, playlist_id):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_several_track_info_by_id(self, track_ids_str):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/tracks?ids={track_ids_str}", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_several_artist_info_by_id(self, artist_ids_str):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/artists?ids={artist_ids_str}", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_several_audio_feature_by_id(self, track_ids_str):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={track_ids_str}", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def destructure_artist_data(self,artists_list):
        artist_infos = []
        artist_genres = []
        artist_popularities = []
        artist_followers = []
        for artist_obj in artists_list:
            artist_infos.append({
                "name": artist_obj["name"],
                "id": artist_obj["id"],
                "genres": artist_obj.get("genres", []),
                "popularity": artist_obj["popularity"],
                "followers": artist_obj["followers"]["total"]
            })
            artist_genres.extend(artist_obj.get("genres", []))
            artist_popularities.append(artist_obj["popularity"])
            artist_followers.append(artist_obj["followers"]["total"])
        return {
            'artist_infos' : artist_infos,
            'artist_genres' : artist_genres,
            'artist_popularities' : artist_popularities,
            'artist_followers' : artist_followers,
        }
    
    def construct_track_info_dict(self,track_json = None, audio_features_json = None, artist_json = None): 
        
        most_popular_artist = max(artist_json['artist_infos'], key=lambda x: x["popularity"])
        
        search_str = f"{track_json['name']} {most_popular_artist['name']}"
        
        lyrics_audio_data = self.yt_scraper.scrap_by_search(search_str=search_str, download_output_file_path = "songs")
        
        track_info = {
        "track_name": track_json["name"],
        "track_id": track_json["id"],
        "track_number": track_json["track_number"],
        "disc_number": track_json["disc_number"],
        "duration_ms": track_json["duration_ms"],
        "explicit": track_json["explicit"],
        "popularity": track_json["popularity"],
        "preview_url": track_json["preview_url"],
        "isrc": track_json["external_ids"].get("isrc"),
        "album_name": track_json["album"]["name"],
        "album_id": track_json["album"]["id"],
        "album_type": track_json["album"]["album_type"],
        "album_total_tracks": track_json["album"]["total_tracks"],
        "album_release_date": track_json["album"]["release_date"],
        "album_release_date_precision": track_json["album"]["release_date_precision"],
        "album_images": track_json["album"]["images"],
        "popular_artist": most_popular_artist["name"],
        "popular_artist_id": most_popular_artist["id"],
        "artist_names": [artist["name"] for artist in track_json["artists"]],
        "artist_ids": [artist["id"] for artist in track_json["artists"]],
        "combined_genres": list(set(artist_json['artist_genres'])),
        "artist_popularity": most_popular_artist["popularity"],
        "artist_followers": most_popular_artist["followers"],
        "external_url": track_json["external_urls"]["spotify"],
        "acousticness": audio_features_json["acousticness"],
        "danceability": audio_features_json["danceability"],
        "energy": audio_features_json["energy"],
        "instrumentalness": audio_features_json["instrumentalness"],
        "key": audio_features_json["key"],
        "liveness": audio_features_json["liveness"],
        "loudness": audio_features_json["loudness"],
        "mode": audio_features_json["mode"],
        "speechiness": audio_features_json["speechiness"],
        "tempo": audio_features_json["tempo"],
        "time_signature": audio_features_json["time_signature"],
        "valence": audio_features_json["valence"],
    }
        track_info.update(lyrics_audio_data)
        return track_info
        
    
    def get_several_track_data(self, track_ids_list : list[str]):
        data = []
        for idx in range(0,len(track_ids_list),50):
            track_id_str = ",".join(track_ids_list[idx:idx+50])
            tracks_json = self.get_several_track_info_by_id(track_id_str)["tracks"]
            audio_features_json = self.get_several_audio_feature_by_id(track_ids_str=track_id_str)["audio_features"]
            
            
            assert len(tracks_json) == len(audio_features_json)
            for track_obj, audio_feat_obj in zip(tracks_json, audio_features_json):
                artist_ids_list = [artist['id'] for artist in track_obj["artists"]]
                artists_id_str = ','.join(artist_ids_list[:50])
                artists_data = self.get_several_artist_info_by_id(artists_id_str)["artists"]
                destructured_artist_data = self.destructure_artist_data(artists_list=artists_data)                
                data.append(self.construct_track_info_dict(track_obj, audio_feat_obj, destructured_artist_data))
        return data
    

[1, 2, 3, 4]

### Creating scraper instance

In [83]:

# Load environment variables
load_dotenv(dotenv_path='.env')
# Setup Spotify API credentials
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_uri = os.getenv("REDIRECT_URL")

In [84]:
scraper = SpotifyScraper(
  client_id=client_id,
  client_secret=client_secret,
  redirect_uri="http://localhost:8888/callback",
)

In [85]:
track_data = scraper.get_several_track_data(['1GEBsLDvJGw7kviySRI6GX'])
print()

Downloaded successfully: songs\LYyUr75fs_o.mp4



In [87]:
pprint(track_data)

[{'acousticness': 0.178,
  'album_id': '3EzFY9Rg0PpbADMth746zi',
  'album_images': [{'height': 640,
                    'url': 'https://i.scdn.co/image/ab67616d0000b27360cb9332e8c8c7d8e50854b3',
                    'width': 640},
                   {'height': 300,
                    'url': 'https://i.scdn.co/image/ab67616d00001e0260cb9332e8c8c7d8e50854b3',
                    'width': 300},
                   {'height': 64,
                    'url': 'https://i.scdn.co/image/ab67616d0000485160cb9332e8c8c7d8e50854b3',
                    'width': 64}],
  'album_name': 'Fearless (Big Machine Radio Release Special)',
  'album_release_date': '2008-11-11',
  'album_release_date_precision': 'day',
  'album_total_tracks': 26,
  'album_type': 'album',
  'artist_followers': 114193835,
  'artist_ids': ['06HL4z0CvFAxyc27GXpf02'],
  'artist_names': ['Taylor Swift'],
  'artist_popularity': 100,
  'audio_file_path': 'songs\\LYyUr75fs_o.mp4',
  'combined_genres': ['pop'],
  'danceability': 0.687,
  

In [60]:
playlists = scraper.get_user_playlists()

Retrieving user playlists...
Playlists retrieved successfully.


In [61]:
tracks = scraper.get_playlist_info_by_id(playlists["items"][0]["id"])

In [80]:
track_ids

['7MJQ9Nfxzh8LPZ9e9u68Fq',
 '1GEBsLDvJGw7kviySRI6GX',
 '0gplL1WMoJ6iYaPgMCL0gX',
 '698ItKASDavgwZ3WjaWjtz',
 '3LlmKSHR3Rs0Y3KHQLAYDk',
 '20R4HfKloPKgXDqU7UKk3x',
 '4RVwu0g32PAqgUiJoXsdF8',
 '6habFhsOp2NvshLv26DqMb',
 '18A7ha5BitZjmdHTCwXFbU',
 '75MNhvTCCKsST3YqqUiU9r',
 '1rfofaqEpACxVEHIZBJe6W',
 '5eGEc27nnhtmcOh6RC890a',
 '5PjdY0CKGZdEuoNab3yDmX',
 '6ocbgoVGwYJhOv1GgI9NsF',
 '1mavQ4WCzXSeL2Dm5DS4GQ',
 '3ebXMykcMXOcLeJ9xZ17XH',
 '36jnG0GLshZiH7oWkOq7gV',
 '3z8h0TU7ReDPLIbEnYhWZb',
 '5SDzo5YMWly9n6hVHvxPwp',
 '0ct6r3EGTcMLPtrXHDvVjc',
 '7BMO7O7ImjV8HNTH74Tshv',
 '561jH07mF1jHuk7KlaeF0s',
 '6or1bKJiZ06IlK0vFvY75k',
 '6M47gaKejso9772SKTa3yH',
 '630sXRhIcfwr2e4RdNtjKN',
 '5QDLhrAOJJdNAmCTJ8xMyW',
 '0lYBSQXN6rCTvUZvg9S0lU',
 '5wANPM4fQCJwkGd4rN57mH',
 '4ZtFanR9U6ndgddUvNcjcG',
 '6RUKPb4LETWmmr3iAEQktW',
 '1P17dC1amhFzptugyAO7Il',
 '2DB4DdfCFMw1iaR6JaR03a',
 '3o9kpgkIcffx0iSwxhuNI2',
 '0QLb1y64s617SAnnDoUZLN',
 '5amAJIEdIVtWYEi4wGp7Fn',
 '5flerg6aEao2VayZezVlgu',
 '39LLxExYz6ewLAcYrzQQyP']

In [64]:
track_ids = [item['track']['id']  for item in tracks['items']]
pprint(len(track_ids))

37


In [65]:
tracks_data = scraper.get_several_track_data(track_ids)
pprint(tracks_data)

[{'acousticness': 0.00922,
  'album_id': '3CjuTytLZz3G9znXt2rJgU',
  'album_images': [{'height': 640,
                    'url': 'https://i.scdn.co/image/ab67616d0000b2733f66b5b49ccea004a5ef0db2',
                    'width': 640},
                   {'height': 300,
                    'url': 'https://i.scdn.co/image/ab67616d00001e023f66b5b49ccea004a5ef0db2',
                    'width': 300},
                   {'height': 64,
                    'url': 'https://i.scdn.co/image/ab67616d000048513f66b5b49ccea004a5ef0db2',
                    'width': 64}],
  'album_name': 'SHADYXV',
  'album_release_date': '2014-11-24',
  'album_release_date_precision': 'day',
  'album_total_tracks': 28,
  'album_type': 'compilation',
  'artist_id': '7dGJo4pcD2V6oG8kP0tJRR',
  'artist_name': 'Eminem',
  'danceability': 0.689,
  'disc_number': 2,
  'duration_ms': 320626,
  'energy': 0.735,
  'explicit': True,
  'external_url': 'https://open.spotify.com/track/7MJQ9Nfxzh8LPZ9e9u68Fq',
  'instrumentalness': 

In [71]:
data_df = pd.DataFrame(tracks_data)

In [75]:
data_df.to_pickle('Sample_data_without_artist.pkl')

In [76]:
test_df = pd.read_pickle('Sample_data_without_artist.pkl')

In [79]:
test_df['album_images'][0]

[{'url': 'https://i.scdn.co/image/ab67616d0000b2733f66b5b49ccea004a5ef0db2',
  'width': 640,
  'height': 640},
 {'url': 'https://i.scdn.co/image/ab67616d00001e023f66b5b49ccea004a5ef0db2',
  'width': 300,
  'height': 300},
 {'url': 'https://i.scdn.co/image/ab67616d000048513f66b5b49ccea004a5ef0db2',
  'width': 64,
  'height': 64}]

In [74]:
data_df['album_images'][0]

[{'url': 'https://i.scdn.co/image/ab67616d0000b2733f66b5b49ccea004a5ef0db2',
  'width': 640,
  'height': 640},
 {'url': 'https://i.scdn.co/image/ab67616d00001e023f66b5b49ccea004a5ef0db2',
  'width': 300,
  'height': 300},
 {'url': 'https://i.scdn.co/image/ab67616d000048513f66b5b49ccea004a5ef0db2',
  'width': 64,
  'height': 64}]

31

In [50]:
track_id = tracks["items"][1]["track"]["id"]
print(track_id)

1GEBsLDvJGw7kviySRI6GX


In [ ]:
pprint(scaper.get_track_info_by_id(track_id)["name"])

'Lose Yourself'


In [ ]:
pprint()

'7MJQ9Nfxzh8LPZ9e9u68Fq'


In [ ]:
import requests, time

class spotifyDict(dict):
    def formatLyrics(self, mode: int = 0): return formatLyrics(self, mode)

def getToken(sp_dc: str, sp_key: str = None) -> dict:
    """
    Generates a Spotify Auth Token for searching lyrics/track IDs.\n
    To obtain the sp_key and sp_dc, please check the README on PyPi.

    Returns:
        dict - {"token": "x", "expiry": 0}\n
        dict - {"status": False, "message": "error message", "data": "data"}
    """
    
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
        "App-Platform": "WebPlayer",
        "content-type": "text/html; charset=utf-8",
    }
    cookies = {"sp_dc": sp_dc}
    if sp_key: cookies["sp_key"] = sp_key
    
    r = requests.get("https://open.spotify.com/get_access_token?reason=transport&productType=web_player", headers=headers, cookies=cookies)
    if r.json()["isAnonymous"] == False: return {"token": r.json()["accessToken"], "expiry": r.json()["accessTokenExpirationTimestampMs"]}
    else: return {"status": False, "message": "Error while trying to grab Spotify token.", "data": r.text}

def checkExpiry(info: dict) -> dict:
    """
    Check the expiry of your token.

    Arguments:
        info {dict} - Information (contains token and expiry)\n
    """
    if info["expiry"] > round(time.time()-3*1000): return {"expired": False}
    return {"expired": True}

def formatLyrics(lyrics: dict, mode: int = 0) -> dict:
    """
    Format the lyrics to your liking with several modes.

    Arguments:
        lyrics {dict} - Takes the direct output from the getLyrics function. Needs the lyrics in the base spotify format.\n
        mode {int} - Choose which mode to format the lyrics in.\n
            - 0: Lyrics Only
            - 1: Milliseconds Start and Lyrics
            - 2: Seconds Start and Lyrics
    """

    if "message" not in lyrics: return {"status": False, "message": "No lyrics were provided or they were not in the correct format."}
    if "status" in lyrics:
        if not lyrics["status"]: return {"status": False, "message": "That JSON data is invalid because the status is set to false."}

    toReturn = []
    if mode == 1 or mode == 2:
        if lyrics["message"]["lyrics"]["syncType"] != "LINE_SYNCED": return {"status": False, "message": "Timestamps do not work on songs that aren't line synced."}
    
    if mode == 0:
        for element in lyrics["message"]["lyrics"]["lines"]:
            if str(element["words"]) == "": continue
            toReturn.append(str(element["words"]).replace("\\", ""))
        return {"status": True, "message": toReturn}
    if mode == 1:
        for element in lyrics["message"]["lyrics"]["lines"]:
            if str(element["words"]) == "": continue
            toReturn.append({"milliseconds": str(element["startTimeMs"]), "line": str(element["words"]).replace("\\", "")})
        return {"status": True, "message": toReturn}
    if mode == 2:
        for element in lyrics["message"]["lyrics"]["lines"]:
            if str(element["words"]) == "": continue
            toReturn.append({"seconds": str(round(int(element["startTimeMs"])/1000, 2)), "line": str(element["words"]).replace("\\", "")})
        return {"status": True, "message": toReturn}

    return {"status": False, "message": "That is not a valid format type."}

def getLyrics(info: dict, trackId: str = None, songName: str = None, proxies: dict = {}) -> spotifyDict:
    """
    Search up lyrics of any given song. Uses trackId first over songName.

    Arguments:
        info {dict} - Information (contains token and expiry)\n
        trackId {string} - The track ID for the Spotify song to search up.\n
        songName {string} - The song name to search up.\n
        proxies {dict} - Supports Socks5 and HTTP/S, example: {"http": "http://1.1.1.1:80", "https": "https://1.1.1.1:443", "http": "socks5://1.1.1.1:443"}

    Returns:
        dict - {"status": False if error, True if success, "message": "Reason why/lyrics"}
    """

    if not trackId and not songName: return {"status": False, "message": "No data was provided to search up."}
    
    if "expiry" and "token" not in info: return {"status": False, "message": "The key 'expiry' or 'value' was not found in the info dict."}

    tempCheck = checkExpiry(info)
    if tempCheck["expired"] == True: return {"status": False, "message": "Cookie has expired."}

    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36",
        "App-Platform": "WebPlayer",
        "Authorization": f"Bearer {info['token']}"
    }

    if songName:
        try: r = requests.get(f"https://api-partner.spotify.com/pathfinder/v1/query?operationName=searchDesktop&variables=%7B%22searchTerm%22%3A%22{songName.replace(' ', '+')}%22%2C%22offset%22%3A0%2C%22limit%22%3A10%2C%22numberOfTopResults%22%3A5%2C%22includeAudiobooks%22%3Afalse%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22130115162add6f3499d2f88ead8a37a7cad1d4d2314f3a206377035e7d26b74c%22%7D%7D", headers=headers, proxies=proxies)
        except Exception as e: return {"status": False, "message": f"Error occured: {e}"}

        try: trackId = r.json()["data"]["searchV2"]["tracksV2"]["items"][0]["item"]["data"]["id"]
        except Exception as e: return {"status": False, "message": f"Error occured: {e} - song doesn't exist?"}

    if trackId:

        try: r = requests.get(f"https://spclient.wg.spotify.com/color-lyrics/v2/track/{trackId}?format=json&market=from_token", headers=headers, proxies=proxies)
        except Exception as e: return {"status": False, "message": f"Error occured: {e}"}

        if '"lines"' in r.text: return spotifyDict({"status": True, "message": r.json()})
        else: return {"status": False, "message": f"Could not find track/Spotify responded with status: {r.status_code}, text: {r.text}"}